## 업비트 API 실습

In [ ]:
import os
import jwt  # PyJWT
import uuid
import hashlib
from urllib.parse import urlencode
import requests

### 환경변수 호출

In [ ]:
from dotenv import load_dotenv

load_dotenv()

ACCESS_KEY = os.environ["UPBIT_OPEN_API_ACCESS_KEY"]
SECRET_KEY = os.environ["UPBIT_OPEN_API_SECRET_KEY"]
SERVER_URL = os.environ["UPBIT_OPEN_API_SERVER_URL"]
SERVER_URL

os.environ을 조회 해 보았을때 .env에서 추가한 변수들이 정상적으로 추가되어 있는지 확인해보자  
만약 정상적으로 로딩이 안된다면 아래 명령어로 .env 파일을 로딩해보자  
`%dotenv {.env파일의 절대 또는 상대경로}/.env`

In [ ]:
os.environ

### 토큰 만들기

In [ ]:
def create_token(query=None):
    """API에 사용될 토큰을 생성"""
    if query is None:
        print("Start create token without query")
        payload = {
            "access_key": ACCESS_KEY,
            "nonce": str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, SECRET_KEY)
        authorization_token = "Bearer {}".format(jwt_token)
        return authorization_token

    # query는 dict 타입입니다.
    print("Start create token without query")
    print(query)
    m = hashlib.sha512()
    m.update(urlencode(query).encode())
    print("url encoded_query")
    print(urlencode(query).encode())
    query_hash = m.hexdigest()

    print("hash of encoded_query")
    print(query_hash)

    payload = {
        "access_key": ACCESS_KEY,
        "nonce": str(uuid.uuid4()),
        "query_hash": query_hash,
        "query_hash_alg": "SHA512",
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorization_token = "Bearer {}".format(jwt_token)
    return authorization_token

### 파라미터 없이 토큰 생성

In [ ]:
create_token()

### 파라미터가 포함된 토큰 생성

In [ ]:
create_token({'market': 'KRW-BTC', 'side': 'bid', 'price': '10000', 'ord_type': 'price'})

### 계좌 정보 조회

In [ ]:
def query_account():
    token = create_token()
    headers = {"Authorization": token}

    res = requests.get(SERVER_URL + "/v1/accounts", headers=headers)
    return res.json()

In [ ]:
query_account()

### 주문하기

In [ ]:
def send_order(query):
    query_string = urlencode(query).encode()
   
    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}
    res = requests.post(SERVER_URL + "/v1/orders", params=query_string, headers=headers)
    return res.json()

### 시장 가격 매수 주문하기

In [ ]:
query = {
    'market': 'KRW-BTC',
    'side': 'bid',
    'price': '10000',
    'ord_type': 'price'
}
send_order(query)

### 시장 가격 매도 주문하기

In [ ]:
query = {
    'market': 'KRW-BTC',
    'side': 'ask',
    'volume': '0.0002',
    'ord_type': 'market'
}
send_order(query)

### 고정 가격 매수 주문하기

In [ ]:
query = {
    'market': 'KRW-BTC',
    'side': 'bid',
    'price': '38760000',
    'volume': '0.0002',
    'ord_type': 'limit'
}
send_order(query)

### 고정 가격 매도 주문하기

In [ ]:
query = {
    'market': 'KRW-BTC',
    'side': 'ask',
    'price': '39760000',
    'volume': '0.0002',
    'ord_type': 'limit'
}
send_order(query)

### 주문조회

In [ ]:
def query_order_list(is_done_state=False):
    query_states = ["wait", "watch"]
    if is_done_state:
        query_states = ["done", "cancel"]

    states_query_string = '&'.join(["states[]={}".format(state) for state in query_states])
    query_string = states_query_string.encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(SERVER_URL + "/v1/orders", params=query_string, headers=headers)

    return res.json()

### 대기 중인 주문 조회

In [ ]:
query_order_list()

### 체결 된 주문 조회

In [ ]:
query_order_list(True)

### 주문 취소하기

In [ ]:
def cancel_order(order_uuid=None):
    if order_uuid is None:
        return

    query = {
        'uuid': order_uuid,
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.delete(SERVER_URL + "/v1/order", params=query, headers=headers)

    return res.json()

### UUID로 주문 최소

In [ ]:
cancel_order('657c9b11-8d63-4db9-b8b9-14094b7992a2')

### 최근 거래 내역 조회

In [ ]:
def query_latest_trade(market):
    """최근 거래 내역 조회
    response:
        trade_date_utc: 체결 일자(UTC 기준), String
        trade_time_utc: 체결 시각(UTC 기준), String
        timestamp: 체결 타임스탬프, Long
        trade_price: 체결 가격, Double
        trade_volume: 체결량, Double
        prev_closing_price: 전일 종가, Double
        change_price: 변화량, Double
        ask_bid: 매도/매수, String
        sequential_id: 체결 번호(Unique), Long
    """
    querystring = {
        "market": market,
        "count":"2"
    }

    response = requests.request("GET", SERVER_URL + "/v1/trades/ticks", params=querystring)

    return response.json()
query_latest_trade('KRW-BTC')